In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=c0006832c8a83bcd1bc3e63869d5fdfbeca0db736f1536d86cb8d5da51e382f9
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Bây giờ bạn có thể truy cập tệp trong Google Drive thông qua đường dẫn '/content/drive/My Drive/'


Mounted at /content/drive


### Triển khai cả ba kernel ###

In [ ]:
from pyspark import SparkContext, SparkConf
import numpy as np

class SVM:
    def __init__(self, kernel='rbf', C=10000.0, max_iter=10000, degree=3, gamma=1):
        self.kernel = {'poly': lambda x, y: np.dot(x, y.T)**degree,
                       'rbf': lambda x, y: np.exp(-gamma*np.sum((y-x[:, np.newaxis])**2, axis=-1)),
                       'linear': lambda x, y: np.dot(x, y.T)}[kernel]
        self.C = C
        self.max_iter = max_iter

    def restrict_to_square(self, t, v0, u):
        t = (np.clip(v0 + t*u, 0, self.C) - v0)[1] / u[1]
        return (np.clip(v0 + t*u, 0, self.C) - v0)[0] / u[0]

    def fit_partition(self, data):
        X, y = zip(*data)
        X = np.array(X)
        y = np.array(y)
        y = y * 2 - 1

        lambdas = np.zeros_like(y, dtype=float)
        K = self.kernel(X, X) * y[:, np.newaxis] * y

        for _ in range(self.max_iter):
            idxM = np.random.randint(0, len(lambdas))
            idxL = np.random.randint(0, len(lambdas))
            Q = K[[[idxM, idxM], [idxL, idxL]], [[idxM, idxL], [idxM, idxL]]]
            v0 = lambdas[[idxM, idxL]]
            k0 = 1 - np.sum(lambdas * K[[idxM, idxL]], axis=1)
            u = np.array([-y[idxL], y[idxM]])
            t_max = np.dot(k0, u) / (np.dot(np.dot(Q, u), u) + 1E-15)
            lambdas[[idxM, idxL]] = v0 + u * self.restrict_to_square(t_max, v0, u)

        idx, = np.nonzero(lambdas > 1E-15)
        support_vectors = X[idx]
        support_y = y[idx]
        support_lambdas = lambdas[idx]
        return support_vectors, support_y, support_lambdas

    def fit(self, data_rdd):
        # Train the SVM on each partition and gather support vectors and lambdas
        fit_results = data_rdd.mapPartitions(lambda partition: [self.fit_partition(partition)]).collect()

        support_vectors, support_y, support_lambdas = zip(*fit_results)
        support_vectors = np.concatenate(support_vectors)
        support_y = np.concatenate(support_y)
        support_lambdas = np.concatenate(support_lambdas)

        # Compute the bias term b
        kernel_values = self.kernel(support_vectors, support_vectors)
        decision_values = np.sum(kernel_values * support_y[:, np.newaxis] * support_lambdas, axis=1)
        b = np.mean(support_y - decision_values)

        # Broadcast support vectors and lambdas
        sc = SparkContext.getOrCreate()
        self.support_vectors_bc = sc.broadcast(support_vectors)
        self.support_y_bc = sc.broadcast(support_y)
        self.support_lambdas_bc = sc.broadcast(support_lambdas)
        self.b_bc = sc.broadcast(b)

    def predict(self, test_rdd):
        support_vectors_bc = self.support_vectors_bc
        support_y_bc = self.support_y_bc
        support_lambdas_bc = self.support_lambdas_bc
        b_bc = self.b_bc

        def predict_partition(data):
            support_vectors = support_vectors_bc.value
            support_y = support_y_bc.value
            support_lambdas = support_lambdas_bc.value
            b = b_bc.value

            X_test = np.array(list(data))
            kernel_values = self.kernel(X_test, support_vectors)
            decision_values = np.sum(kernel_values * support_y * support_lambdas, axis=1) + b
            predictions = [1 if val > 0 else 0 for val in decision_values]
            return iter(predictions)

        predictions = test_rdd.mapPartitions(predict_partition).collect()
        return predictions



In [ ]:

from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.sql.functions import col




# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("SVM Example") \
    .getOrCreate()

# data = spark.read.option("header", "true").csv("/content/data_undersample.csv")
data = spark.read.csv("demo_data.csv", header=True, inferSchema=True)

# Tạo assembler để chuyển đổi dữ liệu thành định dạng mà SVM yêu cầu
feature_cols = data.columns
feature_cols.remove("diabetes")
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
data = assembler.transform(data).select('features', col('diabetes').alias('label'))


# Chia dữ liệu thành tập huấn luyện và kiểm tra
train_rdd, test_rdd = data.rdd.randomSplit([0.7, 0.3], seed=1234)

X_train = train_rdd.map(lambda x: np.array(x[0]))
y_train = train_rdd.map(lambda x: x[1])

X_test = test_rdd.map(lambda x: np.array(x[0]))
y_test = test_rdd.map(lambda x: x[1])


              precision    recall  f1-score   support

         0.0       0.67      0.54      0.60      2578
         1.0       0.61      0.73      0.66      2520

    accuracy                           0.63      5098
   macro avg       0.64      0.63      0.63      5098
weighted avg       0.64      0.63      0.63      5098



In [ ]:
def calculate_accuracy(y_true, y_pred):
    """
    Tính độ chính xác của dự đoán bằng cách so sánh y_pred với y_true.

    Parameters:
        - y_true: List hoặc numpy array chứa nhãn thực tế.
        - y_pred: List hoặc numpy array chứa nhãn dự đoán.

    Returns:
        - Độ chính xác của dự đoán.
    """
    correct = 0
    total = len(y_true)

    for true, pred in zip(y_true, y_pred):
        if true == pred:
            correct += 1

    accuracy = correct / total
    return accuracy

In [ ]:
import itertools
def grid_search(X_train, y_train, X_test, y_test, Cs, gammas):
    best_accuracy = 0
    best_C = None
    best_gamma = None;
    train_cnt = 1
    for C, gamma in itertools.product(Cs, gammas):
        svm = SVM(C=C, gamma = gamma)
        data_rdd = X_train.zip(y_train)
        svm.fit(train_rdd)
        y_pred = svm.predict(X_test)  # Dự đoán trên tập kiểm tra thay vì tập huấn luyện
        accuracy = calculate_accuracy(np.array(y_test.collect()), y_pred)  # Đánh giá độ chính xác trên tập kiểm tra
        print('...Train lan ', train_cnt)
        train_cnt += 1
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_C = C
            best_gamma = gamma

    return best_C, best_gamma, best_accuracy


# Xác định các giá trị của learning_rate và lambda_param
Cs = [10, 100, 1000, 10000, 100000, 1000000]
gammas = [0.0001, 0.001, 0.01, 0.1, 1, 10]

best_C, best_gamma, best_accuracy = grid_search(X_train, y_train, X_test, y_test, Cs, gammas )

...Train lan  1
...Train lan  2
...Train lan  3
...Train lan  4
...Train lan  5
...Train lan  6
...Train lan  7
...Train lan  8
...Train lan  9
...Train lan  10
...Train lan  11
...Train lan  12
...Train lan  13
...Train lan  14
...Train lan  15
...Train lan  16
...Train lan  17
...Train lan  18
...Train lan  19
...Train lan  20
...Train lan  21
...Train lan  22
...Train lan  23
...Train lan  24
...Train lan  25
...Train lan  26
...Train lan  27
...Train lan  28
...Train lan  29
...Train lan  30
...Train lan  31
...Train lan  32
...Train lan  33
...Train lan  34
...Train lan  35
...Train lan  36


In [ ]:
print(best_C)
print(best_gamma)
print(best_accuracy)

100000
0.001
0.6635935661043546


In [ ]:

# Sử dụng mô hình SVM đã huấn luyện
svm = SVM(C=best_C, gamma=best_gamma)
svm.fit(train_rdd)

# Dự đoán trên tập test
y_pred = svm.predict(X_test)

# Chuyển đổi test_rdd thành mảng numpy để so sánh với nhãn thực tế
y_true = np.array(test_rdd.map(lambda x: x[1]).collect())


              precision    recall  f1-score   support

         0.0       0.69      0.60      0.64      2578
         1.0       0.64      0.72      0.68      2520

    accuracy                           0.66      5098
   macro avg       0.66      0.66      0.66      5098
weighted avg       0.66      0.66      0.66      5098



In [ ]:
def calculate_metrics_per_class(y_true, y_pred, class_label):
    """
    Tính toán precision, recall và f1-score cho một lớp cụ thể dựa trên kết quả dự đoán và nhãn thực tế.

    Parameters:
        - y_true: List hoặc numpy array chứa nhãn thực tế.
        - y_pred: List hoặc numpy array chứa nhãn dự đoán.
        - class_label: Nhãn của lớp cần tính toán metric.

    Returns:
        - precision: Precision của lớp cụ thể.
        - recall: Recall của lớp cụ thể.
        - f1_score: F1-score của lớp cụ thể.
    """
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for true, pred in zip(y_true, y_pred):
        if true == class_label and pred == class_label:
            true_positives += 1
        elif true != class_label and pred == class_label:
            false_positives += 1
        elif true == class_label and pred != class_label:
            false_negatives += 1

    precision = true_positives / (true_positives + false_positives + 1e-15)
    recall = true_positives / (true_positives + false_negatives + 1e-15)
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-15)

    return precision, recall, f1_score

# Tính toán các metric cho mỗi lớp
num_classes = 2  # Số lượng lớp
print("Accuracy: ", best_accuracy)
for class_label in range(num_classes):
    precision, recall, f1_score = calculate_metrics_per_class(y_true, y_pred, class_label)
    print("------------Class:", class_label)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1_score)



Accuracy:  0.6635935661043546
------------Class: 0
Precision: 0.6861443116423196
Recall: 0.6012412723041117
F1-score: 0.6408931155675
------------Class: 1
Precision: 0.6379006692497359
Recall: 0.7186507936507937
F1-score: 0.6758723642470604
